# 项目：评估和清理英国电商公司销售数据

## 分析目标

此数据分析的目的是，根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收。

本实战项目的目的在于练习评估数据干净和整洁度，并且基于评估结果，对数据进行清洗，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了一家英国在线零售公司在2010年12月1日至2011年12月9日期间的所有交易情况，涵盖了该公司在全球不同国家和地区的业务数据。该公司主要销售覆盖各个场景的礼品，包括但不限于生日礼品、结婚纪念品、圣诞礼品等等。该公司的客户群体主要包括批发商和个人消费者，其中批发商占据了相当大的比例。

数据每列的含义如下：
- `InvoiceNo`: 发票号码。6位数，作为交易的唯一标识符。如果这个代码以字母“c”开头，表示这笔交易被取消。
- `StockCode`: 产品代码。5位数，作为产品的唯一标识符。
- `Description`: 产品名称。
- `Quantity`: 产品在交易中的数量。
- `InvoiceDate`: 发票日期和时间。交易发生的日期和时间。
- `UnitPrice`: 单价。价格单位为英镑（£）。
- `CustomerID`: 客户编号。5位数，作为客户的唯一标识符。
- `Country`: 国家名称。客户所居住的国家的名称。

## 读取数据

将 `e_commerce.csv` 读取进 `original_data`

In [1]:
import pandas as pd
original_data = pd.read_csv('e_commerce.csv')
original_data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


## 评估数据

我的思路是：  
首先先进行抽样，观察是否有结构性问题。  
其次用 `df.info()` 方法观察一下整体情况，是否存在缺失值等。调查缺失值可以用 `df/s.isnull()`方法，我们也可以提取出来观察有缺失的列。  
重复值也进行一下筛查 `df.duplicated()`  
不一致值用 `s.value_counts()` 方法进行筛查  
无效、错误数据用 `df.describe()`方法进行筛查  

**开始抽样**

In [3]:
original_data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
300172,563193,23181,BULL DOG BOTTLE TOP WALL CLOCK,1,8/12/2011 16:30,16.63,NaN,United Kingdom
151716,549567,21159,MOODY BOY DOOR HANGER,1,4/10/2011 15:07,1.45,13632.0,United Kingdom
214358,555573,22470,HEART OF WICKER LARGE,2,6/5/2011 15:46,2.95,15349.0,United Kingdom
17873,537781,21791,VINTAGE HEADS AND TAILS CARD GAME,1,12/8/2010 12:46,1.25,17341.0,United Kingdom
483675,577506,22568,FELTCRAFT CUSHION OWL,1,11/20/2011 12:40,3.75,13212.0,United Kingdom
370364,569136,22893,MINI CAKE STAND T-LIGHT HOLDER,12,9/30/2011 14:41,0.42,15834.0,United Kingdom
69523,541971,22777,GLASS CLOCHE LARGE,1,1/24/2011 13:48,16.63,NaN,United Kingdom
436163,574090,22467,GUMBALL COAT RACK,6,11/3/2011 9:17,2.55,17442.0,United Kingdom
379213,569680,23293,SET OF 12 FAIRY CAKE BAKING CASES,8,10/5/2011 14:28,0.83,16803.0,United Kingdom
174418,551814,23052,RECYCLED ACAPULCO MAT TURQUOISE,2,5/4/2011 11:45,8.25,18065.0,United Kingdom


由上图可知，该数据不存在结构性问题，但是能看到 **Customer ID** 存在缺失,说明一定是存在内容性问题。

**用info()方法进行观察**

In [5]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


由上图可知，该数据除了存在 **Customer ID** 缺失外，还有 **Description** 存在缺失。可以考虑将缺失行提出来进行观察。  
同时，**Customer ID** 居然是float类型， **InvoiceDate** 居然是object类型，这些都是需要修改的。

**缺失值处理**

**提取缺失 Description 行**

In [6]:
original_data[original_data['Description'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


由上图可知，`Description`有缺失的行，似乎 `UnitPrice = 0`,下面进行验证，看看是否如此。  
同时还能发现`Quantity`居然出现了负数，显然是异常值，待会进行处理。

In [8]:
original_data[(original_data['Description'].isnull() & (original_data['UnitPrice'] != 0))]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


可以发现，我们的猜想正确，而对于单价是0的订单数据，对我们**畅销产品**的挖掘是无效的，因此这部分可以直接考虑进行删除。

**提取缺失 Customer ID 行**

In [10]:
original_data[original_data['CustomerID'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


`CustomerID` 缺失似乎不会影响咱们的分析，但是我发现 `StockCode` 中存在着 `85099B 85099C DOT`。  
这些与数据集介绍的  

> StockCode: 产品代码。**5位数**，作为产品的唯一标识符。

似乎有矛盾。后续来处理此事。（我思考了一下认为5位数字开头也计入合理的话，可以只筛查一下DOT这种）

**缺失值处理完成，关于重复值：**  
订单号由于用户可以一单买很多东西； 产品号由于很多用户可以购买同一个东西； 客户ID由于一名用户可以购买多次；  
所以这仨唯一标识符都可以重复。  
因此不考虑重复值的问题。

**接下来考虑`Country`中的不一致数据（也就这里可能会有了，涉及到用户填写、个人表达类的，之后都得筛一下）**

In [12]:
original_data['Country'].value_counts()

Country
United Kingdom          495266
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
China                      288
Singapore                  229
USA                        218
UK                         211
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United States               73
United Arab Emirates        68


存在着美国的 USA United States 两种写法；英国的 United Kingdom UK U.K.三种写法，之后需要进行修改

**最后考虑异常、错误数据**  
用describe方法看看，同时也需要考虑咱们之前提到的产品号的问题。

In [13]:
original_data['StockCode'].value_counts()

StockCode
85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
21431        1
22275        1
17001        1
90187A       1
72759        1
Name: count, Length: 4070, dtype: int64

打算筛选一下不含有5个数字的订单号，并存疑。

In [14]:
original_data[~original_data['StockCode'].str.contains('^\d{5}')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
45,536370,POST,POSTAGE,3,12/1/2010 8:45,18.00,12583.0,France
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
386,536403,POST,POSTAGE,1,12/1/2010 11:27,15.00,12791.0,Netherlands
1123,536527,POST,POSTAGE,1,12/1/2010 13:04,18.00,12662.0,Germany
1423,536540,C2,CARRIAGE,1,12/1/2010 14:05,50.00,14911.0,EIRE
...,...,...,...,...,...,...,...,...
541540,581498,DOT,DOTCOM POSTAGE,1,12/9/2011 10:26,1714.17,NaN,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541730,581570,POST,POSTAGE,1,12/9/2011 11:59,18.00,12662.0,Germany
541767,581574,POST,POSTAGE,2,12/9/2011 12:09,18.00,12526.0,Germany


In [15]:
original_data[~original_data['StockCode'].str.contains('^\d{5}')]['StockCode'].value_counts()

StockCode
POST            1256
DOT              710
M                571
C2               144
D                 77
S                 63
BANK CHARGES      37
AMAZONFEE         34
CRUK              16
DCGSSGIRL         13
DCGSSBOY          11
gift_0001_20      10
gift_0001_10       9
gift_0001_30       8
DCGS0003           5
gift_0001_50       4
PADS               4
gift_0001_40       3
B                  3
DCGS0069           2
DCGS0070           2
DCGS0076           2
DCGS0072           1
DCGS0074           1
DCGS0057           1
m                  1
DCGS0004           1
DCGS0073           1
DCGS0071           1
DCGS0068           1
DCGS0067           1
DCGS0066P          1
DCGS0055           1
Name: count, dtype: int64

看起来似乎有挺多单号存在**非5位数字**的情况呀~ 可以考虑反应一下！数据集的说明可能存在问题呢！  
接下来再用 describe()方法 进行筛选呀~

In [16]:
original_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


`Quantity` 和 `UnitPrice` 出现了问题耶 当然 `Customer ID` 因为数据类型也参与了计算，后续进行修改

提取 Quantity 看一下

In [17]:
original_data[original_data['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


发现好像 `InvoiceNO` 此时都是 C 开头耶,猜想可能都被取消了。

In [20]:
original_data[(original_data['Quantity'] < 0) & ~(original_data['InvoiceNo']).str.contains('^C')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,NaN,United Kingdom


震惊！居然真的有哎~ 但是似乎这里的单价都是0，不会影响我们的后续挖掘。看看到底是不是这么一回事！

In [22]:
original_data[(original_data['Quantity'] < 0) & ~(original_data['InvoiceNo']).str.contains('^C') & ~(original_data['UnitPrice']==0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


发现和我们想的一样，即确实没有取消的订单，单价都是0！  
那说明`Quantity` < 0 的部分确实可以直接删掉哎~ (要么订单取消，要么单价为0)

接下来来看UnitPrice

In [24]:
original_data[(original_data['UnitPrice'] < 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


发现这俩都是坏账调整，不属于实际交易数据，可以直接删除。

## 清理数据

首先整理一下我们需要干啥：  
- 将 **CustomerID** 类型改为 object; 将 **InvoiceDate** 类型改为 datetime
- 删除 **Description** 缺失部分
- 调整 **U.K. UK** 为 **United Kingdom** 
- 调整 **USA** 为 **United States**
- 删除 **Quantity** 和 **UnitPrice** 小于0的部分

**拷贝一份数据先**

In [25]:
cleaned_data = original_data.copy()
cleaned_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


将 **CustomerID** 类型改为 object; 将 **InvoiceDate** 类型改为 datetime

In [27]:
cleaned_data['CustomerID'] = cleaned_data['CustomerID'].astype(str)

In [29]:
cleaned_data['InvoiceDate'] = pd.to_datetime(cleaned_data['InvoiceDate'])

In [32]:
print(cleaned_data['CustomerID'])
print(cleaned_data['InvoiceDate'])

0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 541909, dtype: object
0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]


In [40]:
cleaned_data['CustomerID'] = cleaned_data['CustomerID'].str.slice(0,-2)
cleaned_data['CustomerID']

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 541909, dtype: object

删除 **Description** 缺失部分

In [41]:
cleaned_data = cleaned_data.dropna(subset=['Description'])
cleaned_data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [42]:
cleaned_data['Description'].isnull().sum()

0

调整 **U.K. UK** 为 **United Kingdom** 

In [46]:
cleaned_data = cleaned_data.replace(['UK','U.K.'],'United Kingdom')
len(cleaned_data[(cleaned_data['Country'] == 'UK')])

0

In [47]:
len(cleaned_data[(cleaned_data['Country'] == 'U.K.')])

0

调整 **USA** 为 **United States**

In [48]:
cleaned_data['Country'] = cleaned_data['Country'].replace('USA','United States')
len(cleaned_data[(cleaned_data['Country'] == 'USA')])

0

删除 **Quantity** 和 **UnitPrice** 小于0的部分
单价为0的部分可以删除

In [49]:
cleaned_data = cleaned_data[cleaned_data['Quantity']>=0]
len(cleaned_data[(cleaned_data['Quantity'] < 0)])

0

In [50]:
cleaned_data = cleaned_data[cleaned_data['UnitPrice']>0]
len(cleaned_data[(cleaned_data['UnitPrice'] <= 0)])

0

## 保存清理后的数据

In [51]:
cleaned_data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [52]:
cleaned_data.to_csv('e_commerce_cleaned.csv')

In [53]:
cleaned_data.to_csv('e_commerce_cleaned_noindex.csv',index=False)